## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [76]:
# checking how balanced or imbalance the dataset  is:

attrition_counts = attrition_df['Attrition'].value_counts()

# Calculate percentages
attrition_percentages = attrition_counts / attrition_counts.sum() * 100

# Create a DataFrame from the counts and percentages
attrition_summary = pd.DataFrame({
    'Attrition': attrition_counts.index,
    'Count': attrition_counts.values,
    'Percentage': attrition_percentages.values
})

print("Attrition Summary Table:")
print(attrition_summary)

Attrition Summary Table:
  Attrition  Count  Percentage
0        No   1233   83.877551
1       Yes    237   16.122449


In [75]:
department_counts = attrition_df['Department'].value_counts()

# Calculate percentages
department_percentages = department_counts / department_counts.sum() * 100

# Create a DataFrame from the counts and percentages
department_summary = pd.DataFrame({
    'Department': department_counts.index,
    'Count': department_counts.values,
    'Percentage': department_percentages.values
})

print("Department Summary Table:")
print(department_summary)

Department Summary Table:
               Department  Count  Percentage
0  Research & Development    961   65.374150
1                   Sales    446   30.340136
2         Human Resources     63    4.285714


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition",'Department']]
print(y_df)

     Attrition              Department
0          Yes                   Sales
1           No  Research & Development
2          Yes  Research & Development
3           No  Research & Development
4           No  Research & Development
...        ...                     ...
1465        No  Research & Development
1466        No  Research & Development
1467        No  Research & Development
1468        No                   Sales
1469        No  Research & Development

[1470 rows x 2 columns]


In [70]:
attrition_values = y_df.groupby("Attrition")
attrition_values.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
5,No,Research & Development
6,No,Research & Development
14,Yes,Research & Development
21,Yes,Sales
24,Yes,Research & Development


In [4]:
y_df.shape

(1470, 2)

In [5]:
attrition_df.columns
# use these columns to create x_list, remove attrition and department column

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [6]:
# Create a list of at least 10 column names to use as X data
X_list = ['Age', 'DistanceFromHome',
       'Education', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked', 
       'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager']

In [7]:

# Create X_df using your selected columns
X_df = attrition_df[X_list]

# Show the data types for X_df
X_df.dtypes


Age                          int64
DistanceFromHome             int64
Education                    int64
EnvironmentSatisfaction      int64
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobSatisfaction              int64
NumCompaniesWorked           int64
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StockOptionLevel             int64
TotalWorkingYears            int64
TrainingTimesLastYear        int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsInCurrentRole           int64
YearsSinceLastPromotion      int64
YearsWithCurrManager         int64
dtype: object

In [18]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df,y_df,random_state=1)


In [9]:
# Initialize LabelEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [17]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# Identify non-numerical columns
non_numerical_cols = X_df.select_dtypes(include=['object']).columns

# Apply LabelEncoder to each non-numerical column
for col in non_numerical_cols:
    X_df[col] = label_encoder.fit_transform(X_df[col])

X_df.head(10)

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,OverTime,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,2,2,94,3,2,4,8,1,...,3,1,0,8,0,1,6,4,0,5
1,49,8,1,3,61,2,2,2,1,0,...,4,4,1,10,3,3,10,7,1,7
2,37,2,2,4,92,2,1,3,6,1,...,3,2,0,7,3,3,0,0,0,0
3,33,3,4,4,56,3,1,3,1,1,...,3,3,0,8,3,3,8,7,3,0
4,27,2,1,1,40,3,1,2,9,0,...,3,4,1,6,3,3,2,2,2,2
5,32,2,2,4,79,3,1,4,0,0,...,3,3,0,8,2,2,7,7,3,6
6,59,3,3,3,81,4,1,1,4,1,...,4,1,3,12,3,2,1,0,0,0
7,30,24,1,4,67,3,1,3,1,0,...,4,2,1,1,2,3,1,0,0,0
8,38,23,3,4,44,2,3,3,0,0,...,4,2,0,10,2,3,9,7,1,8
9,36,27,3,3,94,3,2,3,6,0,...,3,2,2,17,3,2,7,7,7,7


In [11]:
# Show the updated data types for X_df
X_df.dtypes

Age                         int64
DistanceFromHome            int64
Education                   int64
EnvironmentSatisfaction     int64
HourlyRate                  int64
JobInvolvement              int64
JobLevel                    int64
JobSatisfaction             int64
NumCompaniesWorked          int64
OverTime                    int32
PercentSalaryHike           int64
PerformanceRating           int64
RelationshipSatisfaction    int64
StockOptionLevel            int64
TotalWorkingYears           int64
TrainingTimesLastYear       int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsInCurrentRole          int64
YearsSinceLastPromotion     int64
YearsWithCurrManager        int64
dtype: object

In [12]:
X_df[["OverTime"]].value_counts()

OverTime
0           1054
1            416
Name: count, dtype: int64

In [19]:

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
x_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
x_train_scaled = x_scaler.transform(X_train)
x_test_scaled = x_scaler.transform(X_test)



In [20]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder()

# Fit the encoder to the training data
y_train_department_encoded = department_encoder.fit_transform(y_train[['Department']]).toarray()

# Create two new variables by applying the encoder
# to the training and testing data

# Fit and transform the Department column in the training data
y_train_department_encoded = department_encoder.fit_transform(y_train[['Department']]).toarray()
y_test_department_encoded = department_encoder.transform(y_test[['Department']]).toarray()

y_train_department_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [21]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder()

# Fit the encoder to the training data
y_train_attrition_encoded = attrition_encoder.fit_transform(y_train[['Attrition']]).toarray()

# Create two new variables by applying the encoder
# to the training and testing data

# Fit and transform the Attrition column in the training data
y_train_attrition_encoded = attrition_encoder.fit_transform(y_train[['Attrition']]).toarray()
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']]).toarray()

In [22]:
print("Transformed y_train_department shape:", y_train_department_encoded.shape)
print("Transformed y_train_attrition shape:", y_train_attrition_encoded.shape)

Transformed y_train_department shape: (1102, 3)
Transformed y_train_attrition shape: (1102, 2)


## Part 2: Create, Compile, and Train the Model

In [23]:
# Find the number of columns in the X training data
len(X_train.columns)

21

In [34]:
# Create the input layer
input_layer = layers.Input(shape=(21,), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu')(shared_layer1)

In [35]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
hidden_layer_department = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3,activation='softmax', name='department_output')(hidden_layer_department)


In [47]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
hidden_layer_attrition = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2,activation='sigmoid', name='attrition_output')(hidden_layer_attrition)



In [48]:
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])


In [49]:
# Compile the model
model.compile(optimizer='adam', 
              loss= {'department_output': 'categorical_crossentropy',
              'attrition_output': 'binary_crossentropy'},
              metrics = {"department_output": "accuracy",
                         'attrition_output': "accuracy"})

# Summarize the model
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      1,408 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │      8,320 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 32)        │      4,128 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      4,128 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_6[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ dense_9[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 18,149 (70.89 KB)

 Trainable params: 18,149 (70.89 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
import numpy as np

# Convert x_train to float32
x_train = np.array(X_train, dtype=np.float32)

# Convert y_train_department_encoded and y_train_attrition_encoded to int32 or float32
y_train_department_encoded = np.array(y_train_department_encoded, dtype=np.int32)
y_train_attrition_encoded = np.array(y_train_attrition_encoded, dtype=np.int32)


In [53]:
# Train the model
train_model = model.fit(X_train, {"department_output": y_train_department_encoded,
                                  "attrition_output": y_train_attrition_encoded},
epochs=100,
batch_size=32,
validation_split=0.2,
verbose=1)

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - attrition_output_accuracy: 0.8219 - department_output_accuracy: 0.6106 - loss: 1.3493 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335 - val_loss: 1.4341
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8577 - department_output_accuracy: 0.6595 - loss: 1.1683 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.5158 - val_loss: 1.3127
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8602 - department_output_accuracy: 0.6072 - loss: 1.1619 - val_attrition_output_accuracy: 0.8281 - val_department_output_accuracy: 0.6335 - val_loss: 1.4926
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8552 - department_output_accuracy: 0.6644 - loss: 1.1906 - val_attrition_output_accuracy: 0.8326 - val_department_output_accuracy: 0.6335 - val_loss: 1.3283
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2m

In [60]:
model_results = model.evaluate(X_test, 
                               {"department_output": y_test_department_encoded,
                                "attrition_output": y_test_attrition_encoded},
                               verbose=1)


model_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.7991 - department_output_accuracy: 0.5591 - loss: 2.0275  


[1.8610105514526367, 0.820652186870575, 0.6086956262588501]

In [64]:
# Extracting and printing the results

department_accuracy = model_results[1]  
attrition_accuracy = model_results[2]  

print(f"Department Accuracy: {department_accuracy:.4f}")

print(f"Attrition Accuracy: {attrition_accuracy:.4f}")


Department Accuracy: 0.8207
Attrition Accuracy: 0.6087


###  Results Interpretation

**Department Accuracy (0.8207):** This means your model correctly predicts the department of an employee about 82.07% of the time. This is a relatively high accuracy, suggesting that the model is performing well in classifying the department.

**Attrition Accuracy (0.6087):** This indicates that the model accurately predicts whether an employee will leave the company (attrition) approximately 60.87% of the time. This accuracy is lower compared to the department prediction, which might suggest that predicting attrition is more challenging for your model.

# Summary

In the provided space below, briefly answer the following questions.

1. **Is accuracy the best metric to use on this data? Why or why not?** 

Answer: When the dataset is not balanced, accuracy metric may not be the best. While accuracy measures the proportion of correctly predicted instances out of all instances, it can be misleading in imbalanced datasets. For example, in this dataset, 83.8% of people do not leave, while only 16% leave the organization. 
```
Attrition Summary Table:
  Attrition  Count  Percentage
0        No   1233   83.877551
1       Yes    237   16.122449
```

If the attrition data is imbalanced (e.g., very few people leave the company), accuracy might give a misleading impression of the model's performance. 
In such cases, metrics like precision, recall, or the F1 score could provide a more nuanced understanding of the model's effectiveness.

It's important to take into consideration other metrics in order to get a better picture. Alternative metrics to consider when datasets are imbalanced: 

**Precision-Recall Curve:** Helps visualize the trade-off between precision and recall.

**Confusion Matrix:** Provides detailed information on true positives, false positives, true negatives, and false negatives.

**ROC Curve and AUC:** Evaluates the trade-offs between true positive rate and false positive rate.


2. **What activation functions did you choose for your output layers, and why?**

Answer: The softmax activation function was chosen for the department output because it handles multi-class classification by producing probabilities that sum to 1 across the classes. There is a total of 3 departments. 
```
Department Summary Table:
               Department  Count  Percentage
0  Research & Development    961   65.374150
1                   Sales    446   30.340136
2         Human Resources     63    4.285714
```

The sigmoid activation function was chosen for the attrition output (e.g. left the company or not) because it is suitable for binary classification, outputting a probability between 0 and 1.


3. **Can you name a few ways that this model might be improved?**

Answer:

- **Handling Imbalanced Data:** If the attrition data is imbalanced, techniques such as oversampling the minority class, undersampling the majority class, or using weighted loss functions could help.
- For example, in this case, the ratio of employees who leave and stay is imbalanced at approximately 8:2. In this case, we can undersample the majority by reducing the sample of people who leave. 

- We can also use a weight loss function. This approach assigns a higher penalty for misclassifying the minority class. It helps the model focus more on getting the minority class correct.
Weighted Loss Functions are a good choice if you want to keep all your data without duplication or deletion but still make sure the model doesn’t ignore the minority class.


- **Feature Engineering:** Another way to improve the model is by creating more meaningful features that can improve the accuracy.